In [ ]:
# Abstract-text-cleaning:leave numbers!
# remove special characters and stop words (delete word indicate tense; creat a new stopword list)
#(note: tense can help with the classification; protocol : future tense ; compeleted RCT: past tense)
#tokenlization and lemma

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import re
import pickle
import spacy
import numpy as np
from spacy.lang.en import English

In [ ]:
spacy_stopwords = spacy.lang.en.stop_words.STOP_WORDS

In [ ]:
nlp = spacy.load('en')

In [ ]:
nlp.Defaults.stop_words -= {"will","had","was","were","did"}

In [ ]:
new_stopwords = spacy_stopwords

In [ ]:
def preprocess_text(df):
    df.AbstractPlusTitle = df.AbstractPlusTitle.fillna(' ')
    df.AbstractPlusTitle = df.AbstractPlusTitle.str.replace('\n',' ')
    df.AbstractPlusTitle = df.AbstractPlusTitle.str.replace('\r',' ')
    return df

In [ ]:
df = pd.read_pickle('save_path/XX.pkl')
#df_comb = df_comb.reset_index(drop =  True)

In [ ]:
#df = df.reset_index()

In [ ]:
df.head()

,AbstractPlusTitle,AbL
0,qigong tai chi increasingly popular mind body ...,0
1,study aims summarize critically evaluate effec...,0
2,tai chi suggested potential effective interven...,0
3,identify empirical evidence effectiveness tai ...,0
4,breast cancer prevalent cancer women worldwide...,0


In [ ]:
df = df[['AbstractPlusTitle','AbL']]

In [ ]:
preprocess_text(df)

,AbstractPlusTitle,AbL
0,Qigong and Tai Chi are two increasingly popula...,0
1,This study aims to summarize and critically ev...,0
2,Tai chi has been suggested as a potential effe...,0
3,To identify empirical evidence on the effectiv...,0
4,Breast cancer is the most prevalent cancer in ...,0
...,...,...
542,Fibromyalgia is a condition characterised by c...,1
543,Mild cognitive impairment (MCI) is cognitive d...,0
544,There is no sufficient evidence on the effecti...,0
545,Many of the currently available treatments for...,0


In [ ]:
def clean_text(x):
  #x=re.sub(r"\'ll"," will ",x) ----  In academic paper, people tend not to use contraction
  #x=re.sub('[^a-zA-Z0-9]', ' ',x).lower()
  x = re.sub('[^a-zA-Z]', ' ',x).lower()
  x = x.split()
  new_list = []
  for word in x:
    if word not in new_stopwords: 
      new_list.append(word)
  return " ".join(new_list)

In [ ]:
df['AbstractPlusTitle'] = df.AbstractPlusTitle.apply(lambda x:clean_text(x))

In [ ]:
#df_comb[df_comb.index.duplicated(keep = False)]

In [ ]:
X = df.loc[:,'AbstractPlusTitle']

In [ ]:
X

0      qigong tai chi increasingly popular mind body ...
1      study aims summarize critically evaluate effec...
2      tai chi suggested potential effective interven...
3      identify empirical evidence effectiveness tai ...
4      breast cancer prevalent cancer women worldwide...
                             ...                        
542    fibromyalgia condition characterised chronic w...
543    mild cognitive impairment mci cognitive declin...
544    sufficient evidence effectiveness acupuncture ...
545    currently available treatments obsessive compu...
546    chronic fatigue cf unexplained fatigue lasting...
Name: AbstractPlusTitle, Length: 547, dtype: object

In [ ]:
y = df.loc[:,'AbL']

In [ ]:
y

0      0
1      0
2      0
3      0
4      0
      ..
542    1
543    0
544    0
545    0
546    1
Name: AbL, Length: 547, dtype: int64

In [ ]:
df.to_pickle('/content/drive/MyDrive/preprodf_Augm.pkl')

In [ ]:
df1 = pd.read_pickle('/content/drive/MyDrive/preprodf_Augm.pkl')

In [ ]:
df1.AbL.value_counts()

0    312
1    235
Name: AbL, dtype: int64

In [ ]:
#df['Abstract'] = df.Abstract.apply(lambda x: x.split())

In [ ]:
#df['Length'] = df.Abstract.apply(lambda x: len(x))

In [ ]:
#df.describe()

,AbL,Length
count,391.000000,391.000000
mean,0.601023,173.902813
std,0.490315,71.761970
min,0.000000,58.000000
25%,0.000000,138.000000
50%,1.000000,163.000000
75%,1.000000,190.500000
max,1.000000,585.000000




```
# This is formatted as code
```

# DistillBERT

In [ ]:
df = pd.read_pickle('save_path')

In [ ]:
!pip3 install --upgrade tensorflow

Requirement already up-to-date: tensorflow in /usr/local/lib/python3.7/dist-packages (2.4.1)


In [ ]:
!pip install ktrain

     |████████████████████████████████| 25.3MB 129kB/s 
     |████████████████████████████████| 6.8MB 46.6MB/s 
     |████████████████████████████████| 983kB 51.5MB/s 
     |████████████████████████████████| 266kB 56.1MB/s 
     |████████████████████████████████| 1.9MB 42.8MB/s 
     |████████████████████████████████| 1.2MB 53.2MB/s 
     |████████████████████████████████| 471kB 51.2MB/s 
     |████████████████████████████████| 901kB 51.6MB/s 
     |████████████████████████████████| 3.3MB 51.2MB/s 
  Created wheel for ktrain: filename=ktrain-0.26.2-cp37-none-any.whl size=25277794 sha256=33431bf12163f221a41ca783d468e3d12e2249dc34d46fe2d993e5cdca396d15
  Stored in directory: /root/.cache/pip/wheels/67/2e/f1/c72afa08df8b2d984b910dea228902ce81dae4511afe9fafd2
  Created wheel for langdetect: filename=langdetect-1.0.8-cp37-none-any.whl size=993193 sha256=7afcd4e45de97794ecce10778fa1996cc1b9d0d0d765c16a3a749e7c3ae9ebb8
  Stored in directory: /root/.cache/pip/wheels/8d/b3/aa/6d99de9f3841d7d3d4

In [ ]:
df_valid = df.sample(frac = 0.2, random_state = 42)

In [ ]:
#df_valid.to_pickle('/content/drive/MyDrive/AbsTil_valid.pkl')

In [ ]:
df_train = df.drop(df_valid.index)

In [ ]:
df_train

,AbstractPlusTitle,AbL
1,study aims summarize critically evaluate effec...,0
3,identify empirical evidence effectiveness tai ...,0
4,breast cancer prevalent cancer women worldwide...,0
5,post stroke mental disorders psmds post stroke...,0
7,insomnia prevalent significant public health c...,0
...,...,...
541,study participants female fibromyalgia fm were...,0
543,mild cognitive impairment mci cognitive declin...,0
544,sufficient evidence effectiveness acupuncture ...,0
545,currently available treatments obsessive compu...,0


In [ ]:
#df_train.to_pickle('/content/drive/MyDrive/AbsTil_train.pkl')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle

In [ ]:
#df_train = pd.read_pickle('/content/drive/MyDrive/HS651_CAMs_systematic_review_automation/BOW_Distill_BertS/AbsTil_train.pkl')
#df_valid = pd.read_pickle('/content/drive/MyDrive/HS651_CAMs_systematic_review_automation/BOW_Distill_BertS/AbsTil_valid.pkl')

In [ ]:
import tensorflow as tf
from sklearn import metrics
import ktrain
from ktrain import text


In [ ]:
#print(tf.__version__)

In [ ]:
(train, val, preproc) = text.texts_from_df(train_df=df_train, text_column='AbstractPlusTitle', label_columns='AbL',
                                           val_df = df_valid,
                                           maxlen = 512, #length of each doc aligned to 512
                                           preprocess_mode = 'distilbert')

['not_AbL', 'AbL']
   not_AbL  AbL
1      1.0  0.0
3      1.0  0.0
4      1.0  0.0
5      1.0  0.0
7      1.0  0.0
['not_AbL', 'AbL']
     not_AbL  AbL
176      1.0  0.0
77       1.0  0.0
361      1.0  0.0
90       1.0  0.0
368      1.0  0.0



preprocessing train...
language: en
train sequence lengths:
	mean : 191
	95percentile : 313
	99percentile : 517


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 194
	95percentile : 302
	99percentile : 550


In [ ]:
bertmodel = text.text_classifier(name = 'distilbert', train_data = train, preproc=preproc)

Is Multi-Label? False
maxlen is 512



done.


In [ ]:
learner = ktrain.get_learner(model = bertmodel,
                             train_data = train,
                             val_data = val,
                             batch_size = 6)

In [ ]:
learner.fit_onecycle(lr = 2e-5, epochs=10)

In [ ]:
predictor = ktrain.get_predictor(learner.model, preproc)

In [ ]:
predictor.save('/content/drive/MyDrive/DistillBert_TitlePlusAbstract')

In [ ]:
text=df_valid['AbstractPlusTitle']
data=list(text)

In [ ]:
#data

In [ ]:
###predictor.get_classes()

In [ ]:
y_pred = predictor.predict(data, return_proba=True)

In [ ]:
df_valid.AbL = df_valid.AbL.astype('int')
df_train.AbL = df_train.AbL.astype('int')

In [ ]:
y_test = df_valid['AbL']

In [ ]:
fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred[:,1])
auc = metrics.auc(fpr, tpr)
#plt.plot(fpr, tpr)
auc

In [ ]:
#### Alternate metric reports ####

In [ ]:
X_test = df_valid.loc [:,'last500']
X_test[78]

In [ ]:
X_test = df_valid.loc [:,'last500']

# X_test.keys()
X_test_BERT = []
for keys in X_test.keys():
    X_test_BERT.append(X_test[keys])
#X_test_BERT[0]

In [ ]:
y_pred_BERT = predictor.predict(X_test_BERT, return_proba=True)

In [ ]:
#predictor.save('/content/drive/MyDrive/BERT_HS651CAMs_pred')

In [ ]:
y_test = df_valid['AbL']

In [ ]:
y_pred_BERT

In [ ]:
fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred_BERT[:,1])
auc = metrics.auc(fpr, tpr)
auc

In [ ]:
# Load saved model

In [ ]:
saved_predictor = ktrain.load_predictor('/content/drive/MyDrive/BERT_HS651CAMs_pred')

In [ ]:
y_pred_BERT1 = saved_predictor.predict(X_test_BERT, return_proba=True)

In [ ]:
y_pred_BERT1 

In [ ]:
fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred_BERT1[:,1])
auc = metrics.auc(fpr, tpr)
auc